In [ ]:
import numpy as np
from itertools import combinations
import re

import plotly.graph_objects as go
import plotly.figure_factory as ff

import umap

In [2]:
class Frontera:
    
    def __init__(self, X, y, percentil_min, percentil_max,N_points_frontera):
        self.X = X
        self.y = y
        self.percentil_min = percentil_min
        self.percentil_max = percentil_max
        self.N_points_frontera = N_points_frontera
        self.dic_categorias = {}
        self.dic_min_dst = {}
        self.list_dist_median = []
        
        self.dic_categorias_UMAP = {}
        self.dic_min_dst_UMAP = {}
        
        self.Frontier_Point_A_X = {}
        self.Frontier_Point_X_A = {}
        
        self.Frontier_Point = {}
        self.Frontier_Vector = {}
        self.class_vector = {}
        
    def distance(self,x0,x1):
        
        # X=(200,3) ; Y=(150,3)
        # diag(X*X')              <> (200,3) * (3,200) > diag(200,200) > (200,1)            X^2   (X-Y)^2
        # 2*X*Y'                  <>(200,3) * (3,150) > (200,150)                          -2XY
        # ones(N,1)*(diag(Y*Y'))' <> (200,1) * (150,1)' > (200,1) * (1,150) >(200,150)      Y^2
        # M_distance = diag(X*X')-2*X*Y'+ones(N,1)*(diag(Y*Y'))'  %%MATLAB
        M_distance = np.reshape( 
            np.diag(np.dot(x0,x0.T)),(-1,1)) - 2*np.dot(x0,x1.T) + np.dot( np.ones((x0.shape[0],1)), np.reshape(np.diag( np.dot(x1,x1.T)).T,(1,-1)) )
        return M_distance
       
    def get_frontier(self):

        for i in np.unique(self.y): 
            dic_categorias_aux ={'X_'+str(i):self.X[self.y==i]}
            self.dic_categorias.update(dic_categorias_aux)

        categorias_list =  [key for key in self.dic_categorias]
        comb_categories = combinations(categorias_list, 2)
        
        for categories in list(comb_categories):
            
            dist = self.distance(self.dic_categorias.get(categories[0]), self.dic_categorias.get(categories[1]))
            self.list_dist_median.append(np.median(dist))

            # row
            row = np.mean(dist, axis=1)
            select_indices_row = np.where( 
                (row > np.percentile(row, self.percentil_min)) & (row < np.percentile(row, self.percentil_max)) )[0]
            min_dst_row = self.dic_categorias.get(categories[0])[select_indices_row]
            dic_min_dst_aux ={categories[0]+'_with_'+categories[1]:min_dst_row} 
            self.dic_min_dst.update(dic_min_dst_aux)

            #Column
            column = np.mean(dist, axis=0)
            select_indices_column = np.where( 
                (column > np.percentile(column, self.percentil_min)) & (column < np.percentile(column, self.percentil_max)) )[0]
            min_dst_column = self.dic_categorias.get(categories[1])[select_indices_column]
            dic_min_dst_aux ={categories[1]+'_with_'+categories[0]:min_dst_column} 
            self.dic_min_dst.update(dic_min_dst_aux)

            
        list_all_frontier = [key for key in self.dic_min_dst]   
        for i in np.unique(self.y):
            
            my_regex = r"^X_" + str(i)
            p = re.compile(my_regex)
            list_A_with_X = [ s for s in list_all_frontier if p.match(s) ]
            
            my_regex = '.._with_X_' + str(i)
            my_regex = r".*" + str(i) + r'$'
            p = re.compile(my_regex)
            list_X_with_A = [ s for s in list_all_frontier if p.match(s) ]
            
            mtz_A = self.dic_min_dst.get(list_A_with_X[0])
            mtz_X = self.dic_min_dst.get(list_X_with_A[0])
            
            for j in range(1,len(list_A_with_X)):
                
                mtz_A = np.concatenate((mtz_A,self.dic_min_dst.get(list_A_with_X[j])), axis=0)
                mtz_X = np.concatenate((mtz_X,self.dic_min_dst.get(list_X_with_A[j])), axis=0)
                
                
            Front_Point_A_X ={'FrontPoints:(' + str(i) + ',X)' : mtz_A}    
            self.Frontier_Point_A_X.update(Front_Point_A_X)
            Front_Point_X_A ={'FrontPoints:(X,' + str(i) + ')' : mtz_X}    
            self.Frontier_Point_X_A.update(Front_Point_X_A)
            
        ## CALCULAR LOS CENTROIDES EN REGION DE FRONTERA
        for (key_A,value_A), (key_X,value_X) in zip(self.Frontier_Point_A_X.items(), self.Frontier_Point_X_A.items()):
    
            Point_A_X_copy = self.Frontier_Point_A_X.copy()
            Point_X_A_copy = self.Frontier_Point_X_A.copy()
            Front_Point = {}
            points_matriz = np.zeros(shape=(self.N_points_frontera, self.X.shape[1]))
            
            for i in range(0,self.N_points_frontera):
                
                dist = self.distance(Point_A_X_copy.get(key_A), Point_X_A_copy.get(key_X))

                #ROW
                min_dist_A_with_X = np.where( dist==np.min(dist) )[0]
                min_A_with_X = Point_A_X_copy.get(key_A)[min_dist_A_with_X]
                Point_A_X_copy.update({ key_A: np.delete(Point_A_X_copy.get(key_A), min_dist_A_with_X, axis=0)})

                #COLUMN
                min_dist_X_with_A = np.where( dist==np.min(dist) )[1]
                min_X_with_A = Point_X_A_copy.get(key_X)[min_dist_X_with_A]

                Point_X_A_copy.update({ key_X: np.delete(Point_X_A_copy.get(key_X), min_dist_X_with_A, axis=0)})
                
                point_value = (np.mean(min_A_with_X+min_X_with_A,axis=0))/2
                
                if i != 0:
                    closeness_criterion = self.distance(np.reshape(point_value,(-1,self.X.shape[1])), points_matriz) < 0.8          #################
                    
                    if not np.any(closeness_criterion == True):
                        points_matriz[i] = point_value
                        
                else:
                    points_matriz[i] = point_value
                
                if (Point_A_X_copy.get(key_A).shape[0] == 0) or (Point_X_A_copy.get(key_X).shape[0] == 0):
                    break
            
            row_points_matriz = np.sum(points_matriz, axis=1)
            select_indices_points_matriz = np.where( row_points_matriz != 0 )[0]
            points_matriz = points_matriz[select_indices_points_matriz]
            
            Front_Point ={'Frontier:' + key_A.split(':')[1] : points_matriz}    
            self.Frontier_Point.update(Front_Point)
        
        
        ### CALCULAR LOS VECTORES
        for (key_region,region), (key_Frontier,Frontier_Point) in zip(self.dic_categorias.items(), self.Frontier_Point.items()):
        #for A_with_B, Frontier_Point in zip(list_A_with_B,list_Frontier_Point):
                        
            Front_vector = {}
            n_row = 0
            vectors_matriz = np.zeros(shape=(self.Frontier_Point.get(key_Frontier).shape[0], self.X.shape[1]))
            
            #print(A_with_B,'--',Frontier_Point)
            
            for key_Frontier in self.Frontier_Point.get(key_Frontier):

                dist = self.distance(self.dic_categorias.get(key_region), np.reshape(key_Frontier,(-1,self.X.shape[1])) ) 

                #ROW
                min_dist_A_with_B = np.where( dist <= 2)[0]                                                                        #################
                min_A_with_B = self.dic_categorias.get(key_region)[min_dist_A_with_B]        

                vector_value = np.median(min_A_with_B,axis=0)
                #print(dic_min_dst_copy.get(A_with_B))
                vectors_matriz[n_row] = vector_value
               
                n_row += 1
            
            Front_vector ={'Vec_Frontier:(X,' + key_region.split('_')[-1] + ')' : vectors_matriz}    
            self.Frontier_Vector.update(Front_vector) 
            
        for (key_origin,value_origin), (key_vec,value_vector) in zip(self.Frontier_Point.items(), self.Frontier_Vector.items()):
            
            vector_aux = np.concatenate((value_origin,value_vector-value_origin), axis=1)
            class_vec = {'Class_vector:' + key_vec.split(':')[1] : vector_aux}
            self.class_vector.update(class_vec) 
            
        ########################
        ### FILTRAR VECTORES ###
        ########################
        ## Unir todos los Frontier_Point en 1 matriz
        mtz_origin = self.Frontier_Point.get([key for key in self.Frontier_Point][0])
        for j in range( 1, len([key for key in self.Frontier_Point]) ):
            mtz_origin = np.concatenate((mtz_origin,self.Frontier_Point.get([key for key in self.Frontier_Point][j])), axis=0)

        ## Calcular matriz de distancias entre todos los Frontier_Point e imputar diagonal de dist Max
        dist_mtz_origin = self.distance(mtz_origin, mtz_origin)
        dis_max = np.max(dist_mtz_origin)
        indent_max = np.nan_to_num(np.identity(dist_mtz_origin.shape[0])*dis_max)        
        dist_mtz_origin = (dist_mtz_origin + indent_max)
        
        mtz_bool_eucl = dist_mtz_origin < 2
        
        ## Unir todos los vectores en 1 matriz
        
        mtz_vec = self.class_vector.get([key for key in self.class_vector][0])
        for j in range( 1, len([key for key in self.class_vector]) ):
            mtz_vec = np.concatenate((mtz_vec,self.class_vector.get([key for key in self.class_vector][j])), axis=0)
        mtz_vec = mtz_vec[:,self.X.shape[1]:]
        
        # first filter
        dist_cosine = 1-pairwise_distances(mtz_vec, metric="cosine") 
        mtz_bool_cos = dist_cosine <= -0.9
        
        diag = np.triu(np.ones(dist_mtz_origin.shape), 1).T == 1
        
        regla = mtz_bool_cos & mtz_bool_eucl & diag 
        select_indices_1 = np.where( regla==True )[0]
        select_indices_2 = np.where( regla==True )[1]
                
        origin_redu = mtz_origin[select_indices_1] 
        vec_redu = mtz_vec[select_indices_1] 
        
        Front_Point = {'Frontier: Reduct': origin_redu}
        self.Frontier_Point.update(Front_Point)
        
        Front_vector ={'Vec_Frontier: Reduct' : vec_redu+origin_redu}    
        self.Frontier_Vector.update(Front_vector) 

        class_vec = {'Class_vector_Reduct:': np.concatenate((origin_redu,vec_redu), axis=1)}
        self.class_vector.update(class_vec)
        
        ## seconf filter
        dist_cosine_2 = 1-pairwise_distances(vec_redu, metric="cosine") 
        mtz_bool_cos_2 = dist_cosine_2 <= 0.8
        diag_2 = np.triu(np.ones(dist_cosine_2.shape), 1).T == 1
        regla_2 = mtz_bool_cos_2 & diag_2
        print(regla_2)
        
        select_indices_1_1 = np.where( regla_2==True )[0]
        
        origin_redu_2 = origin_redu[select_indices_1_1] 
        vec_redu_2 = vec_redu[select_indices_1_1]

        
        Front_Point = {'Frontier: Full Reduct': origin_redu_2}
        self.Frontier_Point.update(Front_Point)
        
        Front_vector ={'Vec_Frontier: Full Reduct' : vec_redu_2+origin_redu_2}    
        self.Frontier_Vector.update(Front_vector) 

        class_vec = {'Class_vector_Full_Reduct:': np.concatenate((origin_redu_2,vec_redu_2), axis=1)}
        self.class_vector.update(class_vec)
        
        #print('------------')
        #print(select_1)
        #print('------------')
        print()
        print()


        
        self.color_list = [0] * len(self.dic_categorias.keys())
        for i in range( len(self.dic_categorias.keys()) ):
            self.color_list[i] = np.random.randint(0, 1000)
        
            
    def plot_muestra_2D(self, col_1, col_2,include_layout=True):
        
        
        door = True
        next_color = 0
        
        
        for key, value in self.dic_categorias.items(): 

            if door:
                fig = go.Figure(data=[go.Scatter(x=value[:, col_1], y=value[:, col_2],
                                                 mode='markers', 
                                                 name= key,
                                                 marker=dict(
                                                     size=6,
                                                     #color=self.color_list[next_color],                # set color to an array/list of desired values
                                                     colorscale='picnic',   # choose a colorscale
                                                     opacity=0.7)
                                                )])
                door = False
                next_color += 1
                
            else:
                
                fig.add_trace(go.Scatter(x=value[:,col_1], y=value[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         size=6,
                                         #color=self.color_list[next_color],                # set color to an array/list of desired values
                                         colorscale='picnic',   # choose a colorscale
                                         opacity=0.7)
                                    ))
                next_color += 1
              
        if include_layout:
            
            fig.update_layout(
                autosize=False,
                width=600,
                height=600,
                margin=dict(l=0, r=0, b=0, t=10))

            fig.show()
            

    def plot_frontera_2D(self, col_1, col_2):    
        
       
        #self.plot_muestra_2D(col_1, col_2, include_layout=False)
        
        door = True
        next_color = 0
        
        
        for key, value in self.dic_categorias.items(): 

            if door:
                fig = go.Figure(data=[go.Scatter(x=value[:, col_1], y=value[:, col_2],
                                                 mode='markers', 
                                                 name= key,
                                                 marker=dict(
                                                     size=6,
                                                     #color=self.color_list[next_color],                # set color to an array/list of desired values
                                                     colorscale='picnic',   # choose a colorscale
                                                     opacity=0.7)
                                                )])
                door = False
                next_color += 1
                
            else:
                
                fig.add_trace(go.Scatter(x=value[:,col_1], y=value[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         size=6,
                                         #color=self.color_list[next_color],                # set color to an array/list of desired values
                                         colorscale='picnic',   # choose a colorscale
                                         opacity=0.7)
                                    ))
                next_color += 1
        
        for key, value_dst in self.Frontier_Point_A_X.items(): 

            fig.add_trace(go.Scatter(x=value_dst[:,col_1], y=value_dst[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         symbol=220,
                                         size=14,
                                         color=np.random.randint(100),                # set color to an array/list of desired values
                                         #colorscale='Viridis',   # choose a colorscale
                                         opacity=1)
                                    ))
            
        for key, value_dst in self.Frontier_Point_X_A.items(): 

            fig.add_trace(go.Scatter(x=value_dst[:,col_1], y=value_dst[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         symbol=220,
                                         size=14,
                                         color=np.random.randint(100),                # set color to an array/list of desired values
                                         #colorscale='Viridis',   # choose a colorscale
                                         opacity=1)
                                    ))
            
        for key, value_dst in self.Frontier_Point.items(): 

            fig.add_trace(go.Scatter(x=value_dst[:,col_1], y=value_dst[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         symbol=300,
                                         size=50,
                                         color=np.random.randint(100),                # set color to an array/list of desired values
                                         #colorscale='Viridis',   # choose a colorscale
                                         opacity=1)
                                    ))

        fig.update_layout(
            autosize=True,
            width=1200,
            height=750,
            margin=dict(l=10, r=10, b=10, t=20))

        fig.show()
        

    def plot_Vectors(self, col_1, col_2):    
        
        door = True
        next_color = 0
        
        for key, value in self.dic_categorias.items(): 

            if door:
                fig = go.Figure(data=[go.Scatter(x=value[:, col_1], y=value[:, col_2],
                                                 mode='markers', 
                                                 name= key,
                                                 marker=dict(
                                                     size=6,
                                                     #color=self.color_list[next_color],                # set color to an array/list of desired values
                                                     colorscale='picnic',   # choose a colorscale
                                                     opacity=0.7)
                                                )])
                door = False
                next_color += 1
                
            else:
                
                fig.add_trace(go.Scatter(x=value[:,col_1], y=value[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         size=6,
                                         #color=self.color_list[next_color],                # set color to an array/list of desired values
                                         colorscale='picnic',   # choose a colorscale
                                         opacity=0.7)
                                    ))
                next_color += 1

        
            
        for key, value_dst in self.Frontier_Point.items(): 

            fig.add_trace(go.Scatter(x=value_dst[:,col_1], y=value_dst[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         symbol=300,
                                         size=50,
                                         color=np.random.randint(100),                # set color to an array/list of desired values
                                         #colorscale='Viridis',   # choose a colorscale
                                         opacity=1)
                                    ))
            
        for key, value_dst in self.Frontier_Vector.items(): 

            fig.add_trace(go.Scatter(x=value_dst[:,col_1], y=value_dst[:,col_2],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         symbol=2,
                                         size=15,
                                         color=np.random.randint(100),                # set color to an array/list of desired values
                                         #colorscale='Viridis',   # choose a colorscale
                                         opacity=1)
                                    ))
            
        for key, value in self.class_vector.items():
            
            col_1_ = int(col_1+value.shape[1]/2)
            col_2_ = int(col_2+value.shape[1]/2)
            
            locals()['fig_{}'.format(key)] = ff.create_quiver(x=value[:,col_1],  y=value[:,col_2],
                                   u=value[:,col_1_], v=value[:,col_2_],
                           scale=1,
                           arrow_scale=.3,
                           name=key,
                           line_width=1.5)
            
            fig.add_traces(data = locals()['fig_{}'.format(key)].data)

            
        
        fig.update_layout(
            autosize=True,
            width=1200,
            height=750,
            margin=dict(l=10, r=10, b=10, t=20))

        fig.show()        
        
        
        
    def plot_UMAP(self):    
        
        trans = umap.UMAP(random_state=42).fit(self.X)

        for key, value in self.dic_categorias.items():
            value_UMAP = trans.transform(value)
            dic_categorias_aux ={key:value_UMAP}
            self.dic_categorias_UMAP.update(dic_categorias_aux)

        for key, value in self.dic_min_dst.items():
            value_UMAP = trans.transform(value)
            dic_min_dst_aux ={key:value_UMAP}
            self.dic_min_dst_UMAP.update(dic_min_dst_aux)
        
        door = True
        next_color = 0
        
        
        for key, value in self.dic_categorias_UMAP.items(): 

            if door:
                fig = go.Figure(data=[go.Scatter(x=value[:,0], y=value[:,1],
                                                 mode='markers', 
                                                 name= key,
                                                 marker=dict(
                                                     size=6,
                                                     #color=self.color_list[next_color],                # set color to an array/list of desired values
                                                     colorscale='picnic',   # choose a colorscale
                                                     opacity=0.7)
                                                )])
                door = False
                next_color += 1
                
            else:
                
                fig.add_trace(go.Scatter(x=value[:,0], y=value[:,1],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         size=6,
                                         #color=self.color_list[next_color],                # set color to an array/list of desired values
                                         colorscale='picnic',   # choose a colorscale
                                         opacity=0.7)
                                    ))
                next_color += 1
        
        for key, value_dst in self.dic_min_dst_UMAP.items(): 

            fig.add_trace(go.Scatter(x=value_dst[:,0], y=value_dst[:,1],
                                     mode='markers', 
                                     name= key,
                                     marker=dict(
                                         symbol=220,
                                         size=14,
                                         color=np.random.randint(100),                # set color to an array/list of desired values
                                         #colorscale='Viridis',   # choose a colorscale
                                         opacity=1)
                                    ))

        fig.update_layout(
            autosize=True,
            width=800,
            height=600,
            margin=dict(l=10, r=10, b=10, t=20))

        fig.show()